# 모델링 구현

## TF-IDF를 활용한 모델 구현
- TF-IDF를 활용해 문장 벡터를 만듬
- 입력 값에 대해 TF-IDF 값으로 벡터화를 진행하기 때문에 사이킷런의 TfidfVectorizer를 사용
- TfidfVectorizer를 사용하기 위해서는 입력값이 텍스트로 이뤄진 데이터 형태여야 함

In [103]:
import pandas as pd
import os

In [104]:
DATA_IN_PATH = './data_in/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA, header=0, quoting=3)

reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

### TF-IDF 벡터화

In [105]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=0.0, analyzer='char', sublinear_tf=True, ngram_range=(1, 3), max_features=5000)

X = vectorizer.fit_transform(reviews)


### TfidfVectorizer()
- min_df
    - 설정한 값 보다 특정 토큰의 df 값이 더 적게 나오면 벡터화 과정에서 제거한다는 의미

- analyzer
    - 분석하기 위한 기준 단위
    - 'word' 와 'char'로 2가지 옵션을 제공함
    - 'word'의 경우 단어 하나를 단위로 함
    - 'char'의 경우 문자 하나를 단위로 함

- sublinear_tf
    - 문서의 단어 빈도 수(term frequency)에 대한 스무딩(smoothing) 여부를 설정하는 값

- ngram_range
    - 빈도의 기본 단위를 어느 범위의 n-gram으로 설정할 것인지를 보는 인자

- max_features
    - 각 벡터의 최대 길이, 특징의 길이를 설정


- TfidfVectorizer를 생성한 후 fit_transform 함수를 사용해 전체 문장에 대한 특징 벡터 데이터 X를 생성
- TF-IDF로 벡터화한 데이터가 준비 됨

### 학습데이터와 검증 데이터 분리

- 사이킷런 라이브러리에 train_test_split 함수를 활용해 학습 데이터와 검증 데이터를 나누기

In [106]:
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42
TEST_SPLIT = 0.2

y = np.array(sentiments)

X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

- 입력값인 X와 정답 라벨을 넘파이 배열로 만든 y에 대해 적용해서 학습 데이터와 검증 데이터로 나눔
- 비율을 기존 학습 데이터의 20 퍼센트로 설정해 검증 데이터를 만듬

### 모델 선언 및 학습
- 선형 회귀 모델을 만들기 위해 사이킷런 라이브러리에서 지원하는 LogisticRegression 클래스의 객체를 생성
- 이후 이 객체의 fit 함수를 호출하면 데이터에 대한 모델 학습이 진행됨

In [107]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

- class_weight
    - 'balanced'로 설정해서 각 라벨에 대해 균형 있게 학습할 수 있게 함

### 검증 데이터로 성능 평가
- 검증 데이터를 가지고 학습한 모델에 대해 성능을 확인
- 성능평가는 앞서 학습한 객체의 score 함수를 이용하여 측정

In [108]:
print(f"Accuracy: {lgs.score(X_eval, y_eval)}")

Accuracy: 0.8598


- 성능을 평가하는 방법으로는 정확도(Accuracy) 외에도 정밀도(precision), 재현율(recall), f1-score, auc 등 다양한 지표가 있음
- 여기서는 간단하게 정확도만 측정함

### Kaggle에 데이터 제출하기
- 학습한 모델을 이용하여 평가 데이터 결과를 예측하고 파일로 저장하기

In [109]:
# 전처리한 텍스트 형태의 평가 데이터 불러오기
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

In [110]:
test_data

,review,id
0,naturally film main themes mortality nostalgia...,"""12311_10"""
1,movie disaster within disaster film full great...,"""8348_2"""
2,movie kids saw tonight child loved one point k...,"""5828_4"""
3,afraid dark left impression several different ...,"""7186_2"""
4,accurate depiction small time mob life filmed ...,"""12128_7"""
...,...,...
24995,sony pictures classics looking sony got rights...,"""2155_10"""
24996,always felt ms merkerson never gotten role fit...,"""59_10"""
24997,disappointed movie familiar case read mark fuh...,"""2531_1"""
24998,opening sequence filled black white shots remi...,"""7772_8"""


In [111]:
# 해당 데이터를 대상으로 이전에 학습 데이터에 대해 사용했던 객체를 사용해 TF_IDF 값으로 벡터화
testDataVecs = vectorizer.transform(test_data['review'])

- 벡터화할때 평가 데이터에 대해서는 fit을 호출하지 않고 그대로 transform만 호출
- fit의 경우 학습 데이터에 맞게 설정했고, 그 설정에 맞게 평가 데이터도 변환하면 됨
- 이후 이 값으로 예측한 후 예측값을 하나의 변수로 할당하고 출력해서 형태를 확인

In [112]:
test_predicted = lgs.predict(testDataVecs)
print(test_predicted)

[1 0 1 ... 0 1 0]


- Kaggle에 제출하기 위해 데이터프레임 형태로 만들어 CSV 파일로 저장
- 데이터 형식은 id값과 결과값으로 구성되야 함

In [113]:
DATA_OUT_PATH = './data_out/'

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

answer_dataset = pd.DataFrame({
    'id': test_data['id'],
    'sentiment': test_predicted
})
answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_tfidf_answer.csv', mode='w', header=True, index=False, quoting=3)

In [114]:
answer_dataset

,id,sentiment
0,"""12311_10""",1
1,"""8348_2""",0
2,"""5828_4""",1
3,"""7186_2""",0
4,"""12128_7""",1
...,...,...
24995,"""2155_10""",1
24996,"""59_10""",1
24997,"""2531_1""",0
24998,"""7772_8""",1
